    L6 提升

1、AdaBoost

提升二分类训练器

通过对原始数据加权，并不断优化权重，对弱分类器进行提升得到强分类器

强分类器是弱分类器的线性组合

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

导入数据 乳腺癌数据

In [2]:
cancer = load_breast_cancer()

In [3]:
print(cancer.data.shape)
print(cancer.feature_names)
print(set(cancer.target))

(569, 30)
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
{0, 1}


In [4]:
x_data = pd.DataFrame(cancer.data,columns = cancer.feature_names)[0:500]

In [5]:
y_data = cancer.target[0:500]

In [6]:
x_test = pd.DataFrame(cancer.data,columns = cancer.feature_names)[500:]

In [7]:
y_test = cancer.target[500:]

需把y_data中的0替换为-1以便后续计算需要

In [8]:
y_data[y_data==0] = -1.0
y_test[y_test==0] = -1.0

首先建立一个树桩

In [9]:
def split(data , feature , value , judge):
    
    #根据所给的特征和值对数据进行分类
    
    judgeArray = np.ones(data.shape[0])
    
    if judge == 'large':
        #如果比value大，则分为0，即没有癌症
        judgeArray[data[feature] > value] = -1.0
    
    else:
        #如果比value小，则分为0，即没有癌症
        judgeArray[data[feature] <= value] = -1.0
        
    
    return judgeArray

In [10]:
def Stump(x_data , y_data , D):
    
    #建立一个单层决策树，返回：分类器、误差率、分类结果
    
    #D是权重向量
    
    #初始化变量
    Stump = {}
    minErr = np.inf
    stepLen = 20
    bestClass = np.ones(len(y_data))
    
    #树桩
    for x_feature in x_data.columns:
        rangeMin = x_data[x_feature].min()
        rangeMax = x_data[x_feature].max()
        steps = ( rangeMax - rangeMin ) / stepLen #对于连续变量进行分段取值
        
        for step in range(-1,int(stepLen)+1):
            for judge in ['large', 'small']:
                value = rangeMin + float(step) * steps
                predictClass = split(x_data , x_feature , value , judge)
                errArray = np.ones(len(y_data))
                errArray[predictClass == y_data] = 0.0
                err = np.dot(D , errArray)
                
                if err < minErr:
                    minErr = err
                    bestClass = predictClass.copy()
                    
                    Stump['feature'] = x_feature
                    Stump['value'] = value
                    Stump['judge'] = judge
    
    return Stump , minErr , bestClass 

In [11]:
example1 , example2 , example3 = Stump(x_data , y_data , np.ones(len(y_data))/len(y_data))

In [12]:
example1

{'feature': 'mean concave points', 'value': 0.0503, 'judge': 'large'}

这样建立了一个单层决策树：当特征'worst perimeter'的值大于110.6469时预测为0，即没有癌症

In [13]:
def AdaBoostDt(x_data , y_data , iterNum):
    
    #用adaboost提升单层决策树
    
    ClassTree = []
    D = np.ones(len(y_data))/len(y_data)
    weighBestClass = np.zeros(len(y_data))
    i = 1
    
    while 1:
        bestStump , error , bestClass = Stump(x_data , y_data , D)
        α = 1/2 * np.log((1 - error) / (error + 1e-10))
        bestStump['α'] = α
        ClassTree.append(bestStump)
        D = D * np.exp(-α * y_data * bestClass)\
            / np.sum(D * np.exp(-α * y_data * bestClass))
        
        #计算分类器的总误差
        
        weighBestClass += α * bestClass
        grossErr = np.dot(np.sign(weighBestClass) != y_data ,
                          np.ones(len(y_data))/len(y_data))
        i += 1
        if grossErr == 0.0:
            break
        
        if i >= iterNum:
            break
    
    return ClassTree

In [14]:
AdaTree = AdaBoostDt(x_data , y_data , 2000)

然后用训练好的强分类器进行预测

In [15]:
def AdaForecast(x_test , AdaClassTree):
    
    #用分类器进行分类
    
    f = np.zeros(x_test.shape[0])
    
    for i in range(len(AdaClassTree)):
        ClassOutPut = split(x_test , AdaClassTree[i]['feature'],
                  AdaClassTree[i]['value'] , AdaClassTree[i]['judge'])
        f += AdaClassTree[i]['α'] * ClassOutPut
        G = np.sign(f)
    
    return G

输出预测结果：

In [16]:
result1 = AdaForecast(x_test , AdaTree)

预测误差率：

In [19]:
np.dot(result1 != y_test , np.ones(len(y_test))/len(y_test))

0.028985507246376812

    2、梯度提升树（GBDT）

GBDT基于单层的CART树进行提升

In [22]:
from sklearn.datasets import load_boston

导入数据：波士顿房价

In [24]:
boston = load_boston()

In [25]:
print(boston.data.shape)
print(boston.feature_names)
print(set(boston.target))

(506, 13)
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
{5.0, 6.3, 7.2, 8.8, 7.4, 10.2, 11.8, 12.7, 13.6, 14.5, 15.0, 16.5, 17.5, 18.9, 18.2, 20.4, 21.6, 22.9, 21.7, 24.0, 19.9, 26.6, 26.5, 27.5, 23.1, 27.1, 28.7, 24.7, 30.8, 33.4, 34.7, 34.9, 36.2, 35.4, 31.6, 33.0, 38.7, 33.2, 43.8, 41.3, 37.2, 39.8, 42.3, 48.5, 44.8, 50.0, 46.7, 48.3, 44.0, 48.8, 46.0, 10.5, 11.5, 11.0, 12.5, 12.0, 13.5, 13.0, 14.0, 16.6, 16.0, 16.1, 16.4, 17.4, 17.1, 17.0, 17.6, 17.9, 18.4, 18.6, 18.5, 18.0, 18.1, 19.6, 19.4, 19.5, 19.1, 19.0, 20.0, 20.5, 20.9, 20.6, 20.1, 21.0, 21.4, 21.5, 21.9, 21.1, 22.0, 22.5, 22.6, 22.4, 22.1, 23.4, 23.5, 23.9, 23.6, 23.0, 24.1, 24.6, 24.4, 24.5, 25.0, 25.1, 26.4, 27.0, 27.9, 28.0, 28.4, 28.1, 28.5, 28.6, 29.4, 29.9, 29.6, 29.1, 29.0, 30.5, 30.1, 31.1, 31.5, 31.0, 32.5, 32.0, 32.9, 32.4, 32.2, 33.3, 33.8, 33.1, 32.7, 8.4, 34.6, 35.2, 35.1, 10.4, 10.9, 7.0, 36.4, 36.0, 36.5, 36.1, 11.9, 37.9, 37.0, 37.6, 37.3, 13.9, 13.4, 14.4, 14.9, 15.

不同于上一个数据集，这里的房价是一个连续变量，需用回归树进行预测

In [40]:
x_boston = pd.DataFrame(boston.data,columns = boston.feature_names)[0:400]
y_boston = pd.DataFrame(boston.target , columns = {'Y'})[0:400]
x_boston_test = pd.DataFrame(boston.data,columns = boston.feature_names)[400:]
y_boston_test = pd.DataFrame(boston.target , columns = {'Y'})[400:]

In [41]:
boston_data = pd.merge(x_boston , y_boston , on = x_boston.index)
boston_test = pd.merge(x_boston_test , y_boston_test , on = x_boston_test.index)

建立一个单层的CART树

In [52]:
def split(data , feature , value):
    
    #根据所给的特征和值分割数据集
    
    left_data = data[data[feature] > value].drop(feature , axis = 1)
    right_data = data[data[feature] <= value].drop(feature , axis = 1)
    
    return left_data , right_data

In [45]:
def sqr_error(data):
    
    #损失函数：平方误差
    
    return np.var(data.iloc[:,-1]) * len(data.iloc[:,-1])

In [46]:
def best_cut(data):
    
    #寻找最佳的分割特征和值
    
    best_S = np.inf
    s = 0.0
    bestFeature = None
    bestValue = 0.0
    
    for x_feature in data.columns[:-1]:
        for value in list(set(data[x_feature])):    
            left_data , right_data = split(data , x_feature , value)

            s = sqr_error(left_data) + sqr_error(right_data)
            
            if s < best_S:
                bestFeature = x_feature
                bestValue = value
                best_S = s
                
    
    return bestFeature , bestValue

In [58]:
def CART_Stump(data):
    
    #种树桩
    
    #如果所有特征都一样，停止分割
    if len(set(data.iloc[:,-1])) == 1:
        return flowers[data.iloc[:,-1].mode()[0]]
    
    
    feature , value = best_cut(data)
    
    
    cartStump = {}
    left_tree , right_tree = split(data , feature , value)
    y = data.columns[-1]
    large = left_tree[y].mean()
    small = right_tree[y].mean()
    
    cartStump['feature'] = feature
    cartStump['value'] = value
    cartStump['large'] = large
    cartStump['small'] = small
    

    return cartStump , left_tree , right_tree

In [59]:
stump , l , r = CART_Stump(boston_data)

In [65]:
def LossFunction(forcast , real):
    
    #定义损失函数
    
    L = np.sum((real - forcast) ** 2)
    
    return L